# Module 3 Project - Terry Stops
In Terry v. Ohio, a landmark Supreme Court case in 1967-1968, the court found that a police officer was not in violation of the "unreasonable search and seizure" clause of the Fourth Amendment, even though he stopped and frisked a couple of suspects only because their behavior was suspicious. Thus was born the notion of "reasonable suspicion", according to which an agent of the police may e.g. temporarily detain a person, even in the absence of clearer evidence that would be required for full-blown arrests etc. Terry Stops are stops made of suspicious drivers.

# Imports and the Data
Let us see what types of data we have and begin data cleaning by evaluating missing values.

In [32]:
# Necessary imports
import pandas as pd
import numpy as np
import matplotlib as plt
import re
import pickle
from datetime import datetime

In [71]:
# The data
df = pd.read_csv('Terry_Stops.csv')
df.head(8)

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,...,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,-,-1,20140000120677,92317,Arrest,None,7500,1984,M,Black or African American,...,11:32:00,-,-,-,SOUTH PCT 1ST W - ROBERT,N,N,South,O,O2
1,-,-1,20150000001670,32260,Field Contact,None,7539,1963,M,White,...,04:55:00,-,-,-,NaN,N,N,-,-,-
2,-,-1,20150000002451,46430,Field Contact,None,7591,1985,M,Hispanic or Latino,...,01:06:00,-,-,-,WEST PCT 3RD W - MARY,N,N,-,-,-
3,-,-1,20150000002815,51725,Field Contact,None,7456,1979,M,White,...,19:27:00,-,-,-,NORTH PCT 2ND W - NORA,N,N,-,-,-
4,-,-1,20150000002815,51727,Field Contact,None,7456,1979,M,White,...,19:32:00,-,-,-,NORTH PCT 2ND W - NORA,N,N,-,-,-
5,-,-1,20150000002833,52168,Field Contact,None,5966,1969,M,White,...,01:25:00,-,-,-,WEST PCT 3RD W - K/Q RELIEF,N,N,-,-,-
6,-,-1,20150000002856,52559,Field Contact,None,7500,1984,M,Black or African American,...,04:44:00,-,-,-,SOUTH PCT 1ST W - ROBERT,N,N,-,-,-
7,-,-1,20150000002870,52057,Field Contact,None,7592,1983,M,White,...,14:32:00,-,-,-,SOUTH PCT 1ST W - ROBERT,N,N,-,-,-


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40313 entries, 0 to 40312
Data columns (total 23 columns):
Subject Age Group           40313 non-null object
Subject ID                  40313 non-null int64
GO / SC Num                 40313 non-null int64
Terry Stop ID               40313 non-null int64
Stop Resolution             40313 non-null object
Weapon Type                 40313 non-null object
Officer ID                  40313 non-null object
Officer YOB                 40313 non-null int64
Officer Gender              40313 non-null object
Officer Race                40313 non-null object
Subject Perceived Race      40313 non-null object
Subject Perceived Gender    40313 non-null object
Reported Date               40313 non-null object
Reported Time               40313 non-null object
Initial Call Type           40313 non-null object
Final Call Type             40313 non-null object
Call Type                   40313 non-null object
Officer Squad               39780 non-null ob

In [73]:
df.isna().sum()

Subject Age Group             0
Subject ID                    0
GO / SC Num                   0
Terry Stop ID                 0
Stop Resolution               0
Weapon Type                   0
Officer ID                    0
Officer YOB                   0
Officer Gender                0
Officer Race                  0
Subject Perceived Race        0
Subject Perceived Gender      0
Reported Date                 0
Reported Time                 0
Initial Call Type             0
Final Call Type               0
Call Type                     0
Officer Squad               533
Arrest Flag                   0
Frisk Flag                    0
Precinct                      0
Sector                        0
Beat                          0
dtype: int64

<b>At first glance it appears that the Officer's squad is the only category with outright unknown values. For now I will ignore these missing values. However, I will continue to investigate to find missing or odd values.</b> <br><br>
# Distribute unknown ages to each group

In [74]:
df['Subject Age Group'].value_counts()

26 - 35         13372
18 - 25          8400
36 - 45          8357
46 - 55          5155
56 and Above     1919
1 - 17           1853
-                1257
Name: Subject Age Group, dtype: int64

## Create new data frame to replace missing age values 
I will randomly replace the missing age categories.

In [75]:
# List of groups to fill (used 26 - 35 twice since it has double most values)
ages = ['1 - 17', '18 - 25', '26 - 35', '26 - 35', '36 - 45', '46 - 55', '56 and Above']

# Change all missing values to a data frame without missing values
df['Subject Age Group'] = df['Subject Age Group'].apply(lambda x: np.random.choice(ages) if x == '-' else x)

df['Subject Age Group'].value_counts()

26 - 35         13713
18 - 25          8590
36 - 45          8531
46 - 55          5335
56 and Above     2098
1 - 17           2046
Name: Subject Age Group, dtype: int64

# Check other columns for missing values

In [76]:
# Find all inital call types
print(df['Initial Call Type'].nunique())
values = df['Initial Call Type'].unique()
values

161


array(['-', 'SUSPICIOUS STOP - OFFICER INITIATED ONVIEW',
       'SHOTS - IP/JO - INCLUDES HEARD/NO ASSAULT',
       'DIST - IP/JO - DV DIST - NO ASLT',
       'NARCOTICS - VIOLATIONS (LOITER, USE, SELL, NARS)',
       'MVC - HIT AND RUN (NON INJURY), INCLUDES IP/JO',
       'THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS)',
       'SUSPICIOUS PERSON, VEHICLE OR INCIDENT', 'SHOPLIFT - THEFT',
       'ASLT - IP/JO - WITH OR W/O WPNS (NO SHOOTINGS)',
       'MVC - WITH INJURIES (INCLUDES HIT AND RUN)',
       'FIGHT - VERBAL/ORAL (NO WEAPONS)',
       'WEAPN-IP/JO-GUN,DEADLY WPN (NO THRT/ASLT/DIST)',
       'DISTURBANCE, MISCELLANEOUS/OTHER', 'PROWLER - IP/JO',
       'PERSON IN BEHAVIORAL/EMOTIONAL CRISIS',
       'FIGHT - IP - PHYSICAL (NO WEAPONS)', 'AUTO RECOVERY', 'TRESPASS',
       'TRAFFIC STOP - OFFICER INITIATED ONVIEW',
       'BURG - IP/JO - RES (INCL UNOCC STRUCTURES)',
       'ROBBERY - IP/JO (INCLUDES STRONG ARM)',
       'SHOTS -DELAY/INCLUDES HEARD/NO ASSAULT',
       'CARJACKI

## There are 161 different initial call types
I will leave these alone for now.
## Let's look at Call Type

In [77]:
df['Call Type'].value_counts()

911                              17444
-                                12725
ONVIEW                            7182
TELEPHONE OTHER, NOT 911          2747
ALARM CALL (NOT POLICE ALARM)      210
TEXT MESSAGE                         2
PROACTIVE (OFFICER INITIATED)        2
SCHEDULED EVENT (RECURRING)          1
Name: Call Type, dtype: int64

<b>I will make 4 categories:
* 911
* On view
* None
* Other

In [78]:
# Replace values to make 4 categories
df['Call Type'] = df['Call Type'].replace('-', 'None').replace('TELEPHONE OTHER, NOT 911', 'Other')
df['Call Type'] = df['Call Type'].replace('ALARM CALL (NOT POLICE ALARM)', 'Other')
df['Call Type'] = df['Call Type'].replace('TEXT MESSAGE', 'Other').replace('PROACTIVE (OFFICER INITIATED)', 'Other')
df['Call Type'] = df['Call Type'].replace('SCHEDULED EVENT (RECURRING)', 'Other').replace('OnView', 'On_View')
df['Call Type'].value_counts()

911       17444
None      12725
ONVIEW     7182
Other      2962
Name: Call Type, dtype: int64

## Look at Frisk Flag
There are some unknown frisk values. I will remove these since they do not amount to a significant number.

In [79]:
df['Frisk Flag'].value_counts()

N    30983
Y     8853
-      477
Name: Frisk Flag, dtype: int64

In [80]:
df['Frisk Flag'].drop(df[df['Frisk Flag'] == '-'].index, inplace=True)
df['Frisk Flag'].value_counts()

N    30983
Y     8853
Name: Frisk Flag, dtype: int64

## Arrest Flag will be the target

In [81]:
df['Arrest Flag'].value_counts()

N    38707
Y     1606
Name: Arrest Flag, dtype: int64

##  Stop Resolution has 5 categories

In [82]:
df['Stop Resolution'].value_counts()

Field Contact               15758
Offense Report              13860
Arrest                       9814
Referred for Prosecution      728
Citation / Infraction         153
Name: Stop Resolution, dtype: int64

## Let's look at Race

In [83]:
df['Subject Perceived Race'].value_counts()

White                                        19784
Black or African American                    12031
Unknown                                       2026
Hispanic                                      1684
-                                             1345
Asian                                         1246
American Indian or Alaska Native              1212
Multi-Racial                                   809
Other                                          152
Native Hawaiian or Other Pacific Islander       24
Name: Subject Perceived Race, dtype: int64

In [84]:
# Replace dashes (-) with Unknown
df['Subject Perceived Race'] = df['Subject Perceived Race'].replace('-', 'Unknown')
df['Subject Perceived Race'].value_counts()

White                                        19784
Black or African American                    12031
Unknown                                       3371
Hispanic                                      1684
Asian                                         1246
American Indian or Alaska Native              1212
Multi-Racial                                   809
Other                                          152
Native Hawaiian or Other Pacific Islander       24
Name: Subject Perceived Race, dtype: int64

<b>Added dashes to the Unknown categroy.</b>
## Officer's Year of Birth

In [85]:
df['Officer YOB'].describe()

count    40313.000000
mean      1982.277131
std          8.834036
min       1900.000000
25%       1977.000000
50%       1984.000000
75%       1989.000000
max       1997.000000
Name: Officer YOB, dtype: float64

<b>Our officers were born from 1900 to 1997 with a median of 1984</b>
## Weapon Types

In [86]:
df['Weapon Type'].value_counts()

None                                 32565
-                                     5481
Lethal Cutting Instrument             1482
Handgun                                262
Knife/Cutting/Stabbing Instrument      262
Firearm Other                          100
Club, Blackjack, Brass Knuckles         49
Blunt Object/Striking Implement         29
Firearm                                 17
Firearm (unk type)                      15
Other Firearm                           11
Mace/Pepper Spray                       10
Club                                     9
Rifle                                    5
None/Not Applicable                      4
Taser/Stun Gun                           4
Shotgun                                  3
Automatic Handgun                        2
Fire/Incendiary Device                   1
Blackjack                                1
Brass Knuckles                           1
Name: Weapon Type, dtype: int64

### Clean up Weapon Type
Weapons category seems a bit repetitive. If needed, will narrow down to the following categories:
* Firearms
* Cutting Instrument
* Striking Instrument
* None
* Other

In [87]:
# Replace dash with None
df['Weapon Type'] = df['Weapon Type'].replace('-', 'None').replace('None/Not Applicable', 'None')

# Replace all firearms with Firearm
df['Weapon Type'] = df['Weapon Type'].replace('Handgun', 'Firearm').replace('Rifle', 'Firearm')
df['Weapon Type'] = df['Weapon Type'].replace('Firearm Other', 'Firearm').replace('Firearm (unk type)', 'Firearm')
df['Weapon Type'] = df['Weapon Type'].replace('Other Firearm', 'Firearm').replace('Shotgun', 'Firearm')
df['Weapon Type'] = df['Weapon Type'].replace('Automatic Handgun', 'Firearm').replace('Rifle', 'Firearm')

# Replace all cutting/stabbing instruments with Cutting Instrument
df['Weapon Type'] = df['Weapon Type'].replace('Lethal Cutting Instrument', 'Cutting Instrument').replace('Knife/Cutting/Stabbing Instrument', 'Cutting Instrument')

# Replace all types of striking instruments with Striking Instrument
df['Weapon Type'] = df['Weapon Type'].replace('Club, Blackjack, Brass Knuckles', 'Striking Instrument').replace('Blunt Object/Striking Implement', 'Striking Instrument')
df['Weapon Type'] = df['Weapon Type'].replace('Blackjack', 'Striking Instrument').replace('Brass Knuckles', 'Striking Instrument')
df['Weapon Type'] = df['Weapon Type'].replace('Club', 'Striking Instrument')

# Replace odd types of weapons with Other
df['Weapon Type'] = df['Weapon Type'].replace('Mace/Pepper Spray', 'Other').replace('Taser/Stun Gun', 'Other')
df['Weapon Type'] = df['Weapon Type'].replace('Fire/Incendiary Device', 'Other')

df['Weapon Type'].value_counts()

None                   38050
Cutting Instrument      1744
Firearm                  415
Striking Instrument       89
Other                     15
Name: Weapon Type, dtype: int64

## Check to see if IDs are needed

In [88]:
df['Subject ID'].describe()

count    4.031300e+04
mean     1.213040e+09
std      3.046154e+09
min     -1.000000e+00
25%     -1.000000e+00
50%     -1.000000e+00
75%     -1.000000e+00
max      1.275286e+10
Name: Subject ID, dtype: float64

In [89]:
df['Terry Stop ID'].describe()

count    4.031300e+04
mean     1.503510e+09
std      3.722270e+09
min      2.802000e+04
25%      1.572900e+05
50%      3.184670e+05
75%      5.052170e+05
max      1.275287e+10
Name: Terry Stop ID, dtype: float64

At this time I don't believe neither subject ID nor terry stop ID will be needed.
# Convert Reported Time
Convert the time to a time object for easier analysis.

In [90]:
df['Reported Time'].describe()

count        40313
unique        6920
top       03:09:00
freq            50
Name: Reported Time, dtype: object

In [91]:
# Change time from object to time
df['Reported Time'] = pd.to_datetime(df['Reported Time'],format='%H:%M:%S').dt.time
df['Reported Time'].head()

0    11:32:00
1    04:55:00
2    01:06:00
3    19:27:00
4    19:32:00
Name: Reported Time, dtype: object

In [92]:
df['Reported Time'].iloc[0].seconds

AttributeError: 'datetime.time' object has no attribute 'seconds'

<b>The above error shows I've successfully changed the time to a datetime object even though it says just "dtype: object".</b>
<br>Note: documentation says since datetime objects are not one of the usual objects it gets presented as just object even though it registers appropriately in the background.<br><br>
# Make column names pythonic
Take column titles and remove spaces, add underscores, and then change to all lowercase.

In [93]:
# Change all df headers to python style
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()
df.head()

,subject_age_group,subject_id,go_/_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_time,initial_call_type,final_call_type,call_type,officer_squad,arrest_flag,frisk_flag,precinct,sector,beat
0,36 - 45,-1,20140000120677,92317,Arrest,None,7500,1984,M,Black or African American,...,11:32:00,-,-,None,SOUTH PCT 1ST W - ROBERT,N,N,South,O,O2
1,18 - 25,-1,20150000001670,32260,Field Contact,None,7539,1963,M,White,...,04:55:00,-,-,None,NaN,N,N,-,-,-
2,1 - 17,-1,20150000002451,46430,Field Contact,None,7591,1985,M,Hispanic or Latino,...,01:06:00,-,-,None,WEST PCT 3RD W - MARY,N,N,-,-,-
3,1 - 17,-1,20150000002815,51725,Field Contact,None,7456,1979,M,White,...,19:27:00,-,-,None,NORTH PCT 2ND W - NORA,N,N,-,-,-
4,26 - 35,-1,20150000002815,51727,Field Contact,None,7456,1979,M,White,...,19:32:00,-,-,None,NORTH PCT 2ND W - NORA,N,N,-,-,-


# Save cleaned file for use in other notebook pages

In [94]:
# Save
df.to_csv('terry-clean.csv', index = False) 


## Afterthought to check final call types

In [31]:
df.final_call_type.nunique()

193

Too many categories to work with - at least for now.